In [1]:
import sys
sys.path.append("../src")
from mistral_client import run_mistral
from ner_post_processing import parse_entities_promptner, get_token_labels

import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from collections import defaultdict

/home/dachun/miniconda3/envs/mistral-ai-hackathon/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("DFKI-SLT/cross_ner", "politics")

class_labels = dataset["validation"].features["ner_tags"].feature.names
index2label = {i: label for i, label in enumerate(class_labels)}
label2index = {v: k for k, v in index2label.items()}

label2index

{'O': 0,
 'B-academicjournal': 1,
 'I-academicjournal': 2,
 'B-album': 3,
 'I-album': 4,
 'B-algorithm': 5,
 'I-algorithm': 6,
 'B-astronomicalobject': 7,
 'I-astronomicalobject': 8,
 'B-award': 9,
 'I-award': 10,
 'B-band': 11,
 'I-band': 12,
 'B-book': 13,
 'I-book': 14,
 'B-chemicalcompound': 15,
 'I-chemicalcompound': 16,
 'B-chemicalelement': 17,
 'I-chemicalelement': 18,
 'B-conference': 19,
 'I-conference': 20,
 'B-country': 21,
 'I-country': 22,
 'B-discipline': 23,
 'I-discipline': 24,
 'B-election': 25,
 'I-election': 26,
 'B-enzyme': 27,
 'I-enzyme': 28,
 'B-event': 29,
 'I-event': 30,
 'B-field': 31,
 'I-field': 32,
 'B-literarygenre': 33,
 'I-literarygenre': 34,
 'B-location': 35,
 'I-location': 36,
 'B-magazine': 37,
 'I-magazine': 38,
 'B-metrics': 39,
 'I-metrics': 40,
 'B-misc': 41,
 'I-misc': 42,
 'B-musicalartist': 43,
 'I-musicalartist': 44,
 'B-musicalinstrument': 45,
 'I-musicalinstrument': 46,
 'B-musicgenre': 47,
 'I-musicgenre': 48,
 'B-organisation': 49,
 'I-o

In [3]:
fp = "../data/unlabeled/french_knn.csv"
fp_scored = "../data/scored/french_knn.csv"

In [4]:
df = pd.read_csv(fp, header=None, names=["src", "sim1", "sim2", "sim3", "sim4", "sim5"])

In [5]:
sentences = df["sim1"].to_list() + df["sim2"].to_list() + df["sim3"].to_list() + df["sim4"].to_list() + df["sim5"].to_list()
sentences = list(set(sentences))

In [6]:
prompt_french = lambda text: f"""
Dfn: An entity is a person (person), organisation (organisation), french politician (politician), french political party (politicalparty), event (event), election (election), 
country (country), location (location) or other political entity (misc). Dates, times, abstract concepts, adjectives, and verbs are not entities.

Example 1: In the 2014 European Parliament election in France , the National Front won the elections with 24.85 % of the vote , a swing of 18.55 % , winning 24 seats , up from 3 previously .

Answer:
1. 2014 European Parliament election | True | as it is an election (election)
2. France | True | as it is a country (country)
3. National Front | True | as it is a political party (frenchpoliticalparty)

Example 2: The FN received 33.9 % of the votes in the 2017 French presidential election , making it the largest Eurosceptic party in France .

Answer:
1. FN | True | as it is a political party (frenchpoliticalparty)
2. 2017 French presidential election | True | as it is an election (election)
3. Eurosceptic party | True | as it is a political party (frenchpoliticalparty)
4. France | True | as it is a country (country)

Example 3: The 2017 French presidential election caused a radical shift in French politics , as the prevailing parties of The Republicans and Socialists failed to make it to the second round of voting , with far-right Marine Le Pen and political newcomer Emmanuel Macron instead facing each other .

Answer:
1. 2017 French presidential election | True | as it is an election (election)
2. French politics | False | as it is an abstract concept, not a specific entity
3. The Republicans | True | as it is a political party (frenchpoliticalparty)
4. Socialists | True | as it is a political party (frenchpoliticalparty)
5. far-right | False | as it is an adjective describing a political orientation, not a specific entity
6. Marine Le Pen | True | as she is a French politician (frenchpolitician)
7. Emmanuel Macron | True | as he is a French politician (frenchpolitician)
8. political newcomer | False | as it is an abstract concept, not a specific entity

Q. Given the paragraph below, identify a list of possible entities and for each entry explain why it either is or is not an entity.

Paragraph: {text}
"""

In [7]:
scored = defaultdict(list)

for idx, text in enumerate(tqdm(sentences)):
    if (idx + 1) % 100 == 0:
        df_scored = pd.DataFrame(scored)
        df_scored.to_csv(fp_scored, index=False)
    
    try:
        tokens = text.split()
        prompt_input = prompt(text)
        output = run_mistral(prompt_input)
        ner_tags = get_token_labels(text, parse_entities_promptner(output), label2index)

        scored["tokens"].append(text)
        scored["prompt"].append(prompt_input)
        scored["output"].append(output)
        scored["ner_tags"].append(ner_tags)
    except Exception as e:
        print(e)
        continue

df_scored = pd.DataFrame(scored)
df_scored.to_csv(fp_scored, index=False)

  0%|          | 1/683 [00:05<57:39,  5.07s/it]


KeyboardInterrupt: 